Tutorial for MMD with the TorchDrift library: https://towardsai.net/p/machine-learning/drift-detection-using-torchdrift-for-tabular-and-time-series-data

more documentation on TorchDrift MMD: https://torchdrift.org/notebooks/note_on_mmd.html

In [1]:
import pandas as pd
import torch
import torchdrift.detectors as detectors

In [2]:
SAMPLE1 = r'PPO agent 100 alts over 1000+200 2-3-21 results\untargeted_binary_myPGD_03_mask_time_scale_solar_and_consumption_eps_adv_obs-a.csv'
SAMPLE2 = r'PPO agent 100 alts over 1000+200 2-3-21 results\untargeted_binary_myPGD_03_mask_time_scale_solar_and_consumption_eps_adv_obs-a.csv'
SAVE_DIR = r'PPO agent 100 alts over 1000+200 2-3-21 results' + '/'
FILNA = False
NOTNA = False
ATK_NAME = 'untargeted_binary_myPGD_03_mask_time_scale_solar_and_consumption_eps'
if FILNA:
    ATK_NAME += '_fillna'
if NOTNA:
    ATK_NAME += '_notna'
assert(FILNA != NOTNA, ' Why would you use both?!')

<>:11: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:11: SyntaxWarning: assertion is always true, perhaps remove parentheses?
C:\Users\kbrod\AppData\Local\Temp\ipykernel_21084\3679586233.py:11: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(FILNA != NOTNA, ' Why would you use both?!')


##### On the (Statistical) Detection of Adversarial Examples

**Two-sample hypothesis testing** — As stated before, the test we chose is appropriate to handle high dimensional inputs and small sample sizes. We compute the biased estimate of MMD using a **Gaussian kernel**, and then apply **10 000 bootstrapping iterations** to estimate the distributions. Based on this, we compute the **pvalue** and compare it to the threshold, in our experiments **0.05**. For samples of **legitimate data, the observed p-value should always be very high**, whereas for sample sets containing adversarial examples, we expect it to be low—since they are sampled from a different distribution and thus the hypothesis should be rejected. The test is more likely to detect a difference in two distributions when it considers samples of large size (i.e., the sample contains more inputs from the distribution).

In [3]:
BOOTSTRAP = 10_000
PVAL = 0.05
kernel = detectors.mmd.GaussianKernel()

Because our dataset is a time series, we will use MMD on different time segments rather than shuffling the dataset

Load unperturbed observations from untargeted adversarial attack

In [4]:
df_adv_obs = pd.read_csv(SAMPLE1, 
                        index_col=0,
                        dtype='float32',
                        )
df_adv_obs.set_index(df_adv_obs.index.astype(int), inplace=True) #all data is loaded as float32, but the index should be an int

Remove actions if stored in df

In [5]:
if 'a' in df_adv_obs.columns:
    df_adv_obs.drop(columns=['a'], inplace=True)

Load perturbed observations from untargeted adversarial attack (100% adversarial)

In [6]:
df_adv_perturbed_obs = pd.read_csv(SAMPLE2,
                                   index_col=0,
                                   dtype='float32')
df_adv_perturbed_obs.set_index(df_adv_perturbed_obs.index.astype(int), inplace=True) 

In [7]:
if 'a' in df_adv_perturbed_obs.columns:
    df_adv_perturbed_obs.drop(columns=['a'], inplace=True)

Replace the nan, where there was no successful adv sample, with the clean samples. This does change the result, as the test can detect pure adv samples when not diluted with clean ones

In [8]:
if FILNA:
    df_adv_perturbed_obs.fillna(df_adv_obs, inplace=True)

index of the valid entries in the adv observations, used to get the corresponding values from the clean obs

In [9]:
valid_adv_mask = df_adv_perturbed_obs.dropna().index

In [10]:
if NOTNA: #only use clean samples with corresponding adv sample
    valid_adv_mask = df_adv_perturbed_obs.dropna().index
else: #same as dropna, and MMD doesn't handle NaNs anyway
    valid_adv_mask = df_adv_obs.dropna().index

to use mask: df_adv_obs.loc[valid_adv_mask] below

In [11]:
result = detectors.kernel_mmd(torch.from_numpy(df_adv_obs.loc[valid_adv_mask].values).to('cuda'), #clean obs from adv trace
                                  torch.from_numpy(df_adv_perturbed_obs.dropna().values).to('cuda'), #perturbed obs from adv trace
                                  n_perm=BOOTSTRAP,
                                  kernel=kernel)
torch.cuda.empty_cache() #free gpu memory
print(f'mmd:{result[0]}, p-value:{result[1]}')

mmd:0.0001423358917236328, p-value:1.0


convert cuda tensors to numpy

In [12]:
cpu_result = [tensor.item() for tensor in result]

In [13]:
mmd_savename = SAVE_DIR+'MMDs.csv'
try:
    df_mmd = pd.read_csv(mmd_savename,
                         index_col=0)
    df_mmd = df_mmd.append(
                pd.Series(cpu_result,
                        index=df_mmd.columns,
                        name=ATK_NAME,),
            )
    #df_mmd.loc[ATK_NAME] = cpu_result
    df_mmd.to_csv(mmd_savename)
    print(f'{mmd_savename} updated')
except:
    df_mmd = pd.DataFrame([cpu_result],
                      columns=['MMD','p_value'],
                      index=[ATK_NAME])
    df_mmd.to_csv(mmd_savename)
    print(f'{mmd_savename} created')

PPO agent 100 alts over 1000+200 2-3-21 results/MMDs.csv created


Does removing the SOC value change the MMD result?

In [16]:
result = detectors.kernel_mmd(torch.from_numpy(df_adv_obs.loc[valid_adv_mask].drop(columns='electrical_storage_soc').values).to('cuda'), #clean obs from adv trace
                                  torch.from_numpy(df_adv_perturbed_obs.drop(columns='electrical_storage_soc').dropna().values).to('cuda'), #perturbed obs from adv trace
                                  n_perm=BOOTSTRAP,
                                  kernel=kernel)
torch.cuda.empty_cache() #free gpu memory
print(f'mmd:{result[0]}, p-value:{result[1]}')

mmd:0.0007879734039306641, p-value:0.9835000038146973


Nope